<a href="https://colab.research.google.com/github/GyeeunJeong/nlp_with_pytorch_examples/blob/master/CH3_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Hello 파이토치 예제

In [0]:
import torch
import torch.nn as nn

import numpy as np

import random

## 3.3.1 텐서

In [0]:
x1 = torch.Tensor([[1,2], [3,4]])
x2 = torch.from_numpy(np.array([[1,2],[3,4]]))

x3 = np.array([[1,2],[3,4]])

In [4]:
x1, x2, x3

(tensor([[1., 2.],
         [3., 4.]]), tensor([[1, 2],
         [3, 4]]), array([[1, 2],
        [3, 4]]))

## 3.3.2 Autograd
* 자동으로 미분 및 역전파 수행


In [0]:
x = torch.FloatTensor(2, 2)
y = torch.FloatTensor(2, 2)
y.requires_grad_(True)

z = (x+y) + torch.FloatTensor(2, 2)

In [9]:
z

tensor([[3.1385e-35, 0.0000e+00],
        [4.4842e-44, 0.0000e+00]], grad_fn=<AddBackward0>)

In [0]:
with torch.no_grad():
  z = (x+y) + torch.FloatTensor(2,2)

In [11]:
z

tensor([[3.1384e-35, 0.0000e+00],
        [5.0447e-44, 0.0000e+00]])

## 3.3.3 피드포워드

In [0]:
def linear(x, W, b):
  y = torch.mm(x, W) + b
  return y

x = torch.FloatTensor(16,10)
W = torch.FloatTensor(10,5)
b = torch.FloatTensor(5)

y = linear(x, W, b)

In [17]:
y

tensor([[        nan,         nan,         nan,         nan,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,         inf,         inf,         inf,         nan],
        [        inf,    

## 3.3.4 nn.Module
* nn.Module 클래스를 사용해 피드포워드 구현

In [0]:
class MyLinear(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()

    #self.W = nn.Parameter(torch.FloatTensor(input_size, output_size), requires_grad=True)
    #self.b = nn.Parameter(torch.FloatTensor(output_size), requires_grad=True)
    self.linear = nn.Linear(input_size, output_size)

  def forward(self, x):
    #y = torch.mm(x, self.W) + self.b
    y = self.linear(x)
    return y

In [0]:
x = torch.FloatTensor(16,10)
linear = MyLinear(10,5)
y = linear(x)

In [24]:
linear

MyLinear(
  (linear): Linear(in_features=10, out_features=5, bias=True)
)

## 3.3.5 역전파 수행
* 피드포워드를 통해 얻은 값과 실제 정답값과의 차이를 계산해서 오류를 뒤로 전달(back-propagation)

In [0]:
objective = 100

loss = (objective - y.sum())**2

loss.backward()

In [28]:
loss

tensor(nan, grad_fn=<PowBackward0>)

## 3.3.6 train()과 eval()
* 훈련/추론모드
* 기본 : 훈련모드
* eval을 통해 추론모드로 변경하면 드롭아웃, 배치 정규화와 같은 학습과 추론 시 서로 다른 forward() 동작을 하는 모듈들에 대해서도 각 상황에 따라 올바르게 동작
* 추론 끝나면 다시 train() 선언

In [0]:
linear.eval()
linear.train()

## 3.3.7 선형회귀분석 예제

In [0]:
class MyModel(nn.Module):
  def __init__(self, input_size, output_size):
    super(MyModel, self).__init__()

    self.linear = nn.Linear(input_size, output_size)

  def forward(self, x):
    y = self.linear(x)

    return y

# 실제 정답
def ground_truth(x):
  return 3*x[:,0]+x[:,1]-2*x[:,2]

def train(model, x, y, optim):
  # initialize gradients in all parameters in module.
  optim.zero_grad()

  # feed-forward
  y_hat = model(x) # y근사값
  # get error between answer and inferenced
  loss = ((y-y_hat)**2).sum() / x.size(0)

  # back-propagation
  loss.backward()

  # one-step of gradient descent
  optim.step()

  return loss.data

In [0]:
# hyperparameter
batch_size = 1
n_epochs = 1000
n_iter = 10000

model = MyModel(3,1)
optim = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.1)

In [48]:
model

MyModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

In [49]:
for epoch in range(n_epochs):
  avg_loss = 0

  for i in range(n_iter):
    x = torch.rand(batch_size, 3)
    y = ground_truth(x.data)
    loss = train(model, x, y, optim)

    avg_loss += loss
    avg_loss = avg_loss / n_iter

  # simple test sample to check the network
  x_valid = torch.FloatTensor([[.3, .2, .1]])
  y_valid = ground_truth(x_valid.data)

  model.eval()
  y_hat = model(x_valid)
  model.train()

  print(avg_loss, y_valid.data[0], y_hat.data[0,0])

  if avg_loss < .001: # finish the training if the loss is smaller than .001.
    break

tensor(7.6221e-06) tensor(0.9000) tensor(0.4846)


## 3.3.8 GPU 사용하기
* cuda() 함수를 통해 원하는 객체를 GPU 메모리에 복사/이동
* cpu() 함수를 통해 다시 PC의 메모리로 복사/이동
* to() 함수를 통해 텐서/모듈을 원하는 디바이스로 이동

In [0]:
# Note that tensor is declared in torch.cuda
x = torch.cuda.FloatTensor(16, 10)
linear = MyLinear(10,5)
linear.cuda()
y = linear(x)